# Material Cost Demo

In this Notebook is demonstrated how procurement cost can be added which result in an estimate of the total procurement of the cross-section

In [1]:
import breakwater as bw

c:\users\p17\documents\github\breakwater\breakwater\rubble_2D.py:1298: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if layer is 'core':
c:\users\p17\documents\github\breakwater\breakwater\rubble_2D.py:1303: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  elif (self._input_arguments['armour'] is not 'Rock'
c:\users\p17\documents\github\breakwater\breakwater\rubble_2D.py:1304: SyntaxWarning: "is" with a literal. Did you mean "=="?
  and layer is 'armour'):
c:\users\p17\documents\github\breakwater\breakwater\rubble_2D.py:1321: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if output is 'variant' or output is 'average':
c:\users\p17\documents\github\breakwater\breakwater\rubble_2D.py:1321: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if output is 'variant' or output is 'average':
c:\users\p17\documents\github\breakwater\breakwater\rubble_2D.py:1327: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif output is 'layer':
c:\users\p17\d

In [2]:
# hydraulic conditions

h = 27 
Hm0 = 10
h = 25
Tp = 15
N = int(8 * 3600 / Tp)
slope_foreshore = (1, 100)
DHWL = 0.2

# damage levels

Sd = 4
Nod = 3
q = 100

# densities

rho_w = 1025  
rho_rock = 2650
rho_concrete = 2500

In [3]:
# create the limit state

battjes = bw.BattjesGroenendijk(Hm0=Hm0, h= (h + DHWL), slope_foreshore=slope_foreshore)
H2_per = battjes.get_Hp(0.02)

uls = bw.LimitState(
    h=(h + DHWL), H2_per=H2_per, q=q, Hm0=Hm0, Tp=Tp, Sd=Sd, Nod=Nod, label="uls"
)
uls.transform_periods(0.5)

In [4]:
# create cubipod armour units

units = {
    1: {"D": 1, "h": 1, "Vc": 0.59},
    1.5: {"D": 1.15, "h": 1.10, "Vc": 0.68},
    3: {"D": 1.43, "h": 1.40, "Vc": 0.85},
    5.1: {"D": 1.72, "h": 1.7, "Vc": 1.02},
    8.1: {"D": 2.01, "h": 2.0, "Vc": 1.118},
    12.1: {"D": 2.29, "h": 2.3, "Vc": 1.35},
    23.6: {"D": 2.87, "h": 2.9, "Vc": 1.69},
    46.10: {"D": 3.59, "h": 3.6, "Vc": 2.12},
    79.70: {"D": 4.3, "h": 4.3, "Vc": 2.54},
}

concrete_units = bw.ConcreteArmour(
    kd=12,
    name="Cubipods",
    rho= rho_concrete,
    units=units,
)

In [5]:
# create a standard rock grading

grading = bw.RockGrading(rho=rho_rock)

In [6]:
# add EUR cost rock only

grading_costs = {'LMA_5/40': {'cost': 20, 'CO2': None}, 'LMA_10/60': {'cost': 30, 'CO2': None}, 
                    'LMA_40/200': {'cost': 40, 'CO2': None}, 'LMA_15/300': {'cost': 50, 'CO2': None}, 
                    'LMA_60/300': {'cost': 60, 'CO2': None}, 'HMA_300/1000': {'cost': 70, 'CO2': None}, 
                    'HMA_1000/3000': {'cost': 80, 'CO2': None}, 'HMA_3000/6000': {'cost': 90, 'CO2': None}, 
                    'HMA_6000/10000': {'cost': 100, 'CO2': None}, 'HMA_10000/15000': {'cost': 110, 'CO2': None}}

grading.add_cost(grading_costs)

In [7]:
# create a design space

structure_types = ["RRM", "CRM"]
slope_range = (2, 3)
crest_width_range = (8.61, 25, 20)
Dn50_core_range = (0.2, 0.5, 3)

In [8]:
# run configurations

configs = bw.Configurations(
    structure= ['CRMR'],
    LimitState= uls,
    rho_w= rho_w,
    slope_foreshore= slope_foreshore,
    Grading= grading,
    slope= slope_range,
    B= crest_width_range,
    core_material= {'class': 'quarry', 'Dn50': 0.5},
    N= N,
    ArmourUnit= concrete_units,
    filter_rule= "Xbloc",
    safety= 0,
    phi= 40,
)

Computing Rubble Mound [=====================] 20/20


In [9]:
configs.show_warnings()

| Warning                                        |   Count |
|------------------------------------------------|---------|
| used Hm0 instead of H13 from uls               |      20 |
| Cubipods is not supported, continued with kt=1 |      20 |


In [10]:
# armour unit costs

unit_cost = {
    1: {'cost': 20, 'CO2': None},
    1.5: {'cost': 30, 'CO2': None},
    3: {'cost': 40, 'CO2': None},
    5.1: {'cost': 50, 'CO2': None},
    8.1: {'cost': 60, 'CO2': None},
    12.1: {'cost': 70, 'CO2': None},
    23.6: {'cost': 80, 'CO2': None},
    46.10: {'cost': 90, 'CO2': None},
    79.70: {'cost': 100, 'CO2': None},
}

In [11]:
# add cost

configs.add_cost(
    core_price= {'cost': 10, 'CO2': None},
    fill_price= {'cost': 5, 'CO2': None},
    unit_price= unit_cost,
)

In [12]:
# params we can select in the design explorer

configs.df.columns

Index(['B', 'B_toe', 'Dn50_core', 'concept', 'id', 'slope', 'slope_toe',
       'type', 'warnings', 'material_cost', 'material_CO2'],
      dtype='object')

In [13]:
to_DE = False

if to_DE:
    configs.to_design_explorer(
        params=["slope", "class armour", "B", "Rc", "computed Dn50 armour", "material_cost"]
    )